CA02: This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm. 

In this assignment you will ...
1. Complete the code such a way that it works correctly with this given parts of the program.
2. Explain as clearly as possible what each part of the code is doing. Use "Markdown" texts and code commenting to explain the code

IMPORTANT NOTE:

The path of your data folders 'train-mails' and 'test-mails' must be './train-mails' and './test-mails'. This means you must have your .ipynb file and these folders in the SAME FOLDER in your laptop or Google Drive. The reason for doing this is, this way the peer reviewes and I would be able to run your code from our computers using this exact same relative path, irrespective of our folder hierarchy.

In [8]:
#Import all of the necessary packages in order to run the model
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from google.colab import drive

In [9]:
#This is creating a function that will first create a dictionary and then go through and split
#all of the words and characters, find the ones necessary of removal, and then find the 3000 most common ones
def make_Dictionary(root_dir):
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words #for loop that goes through and uses a space to split all characters and words and append to dictionary
  dictionary = Counter(all_words)  
  list_to_remove = list(dictionary) #turns the dictionary to a list

  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item] #removes any nonalpha charcaters from the dictionary
    elif len(item) == 1:
      del dictionary[item] #removes any words that are only a single letter
  dictionary = dictionary.most_common(3000) #finds the 3000 most common words in the dictionary
  return dictionary            

In [10]:
#This defines a function that populates a feature matrix of 3000 columns that goes through each email and populates
#the value, then it looks at the name of each email to decide if its spam based on the name

def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000)) #matrix based on the 3000 most common words
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split() #for loop to enumerate each of the items in the dictionary
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word) #creates a matrix with the word frequency
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1 #uses the file name to determine if it has spam
  return features_matrix, train_labels                

In [11]:
# this mounts the drive and recalls the files of our two train and test datasets
drive.mount('/content/drive')
TRAIN_DIR = '/content/drive/My Drive/CA02/train-mails'
TEST_DIR = '/content/drive/My Drive/CA02/test-mails'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#Trains our model using Naive Bayes algorithm, to then predict, and evaluate our preformance
dictionary = make_Dictionary(TRAIN_DIR) #recalls the function above and uses it on out train dataset

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR) #populate the model
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB() #use Gaussian Naive Bayes model

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels) #train the model using Gaussian
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix) #predicts based on the model completed above
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels)) #print the accuracy of the model to evaluate its preformance

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9655172413793104


======================= END OF PROGRAM =========================